# Convolutional Neural Networks

In [1]:
import math
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## ConvLayerOutputShapeCalculator

In [2]:
class ConvLayerOutputShapeCalculator:
    
    def compute_conv_output_shape(self, 
                                  input_shape, 
                                  filter_shape, 
                                  n_filters, 
                                  stride, 
                                  padding_size):

        print('Input Shape: {}'.format(input_shape))
        print('Filter Shape: {}'.format(filter_shape))
        print('Num Filters: {}'.format(n_filters))
        print('')
        
        conv_h = self.compute_conv_height(input_shape[0], filter_shape[0], padding_size, stride)
        conv_w = self.compute_conv_width(input_shape[1], filter_shape[1], padding_size, stride)
        conv_d = n_filters
        
        print('Conv Height: {}'.format(conv_h))
        print('Conv Width: {}'.format(conv_w))
        print('Conv Depth: {}'.format(conv_d))
        print('')
        
        return (conv_h, conv_w, conv_d)
    
    def compute_conv_height(self, input_h, filter_h, p, stride):
        return int((input_h - filter_h + 2*p) / stride) + 1
    
    def compute_conv_width(self, input_w, filter_w, p, stride):
        return int((input_w - filter_w + 2*p) / stride) + 1

In [4]:
input_shape = [32, 32, 3]
filter_shape = [8, 8, 3]
n_filters = 20
stride = 2
padding_size = 1

calculator = ConvLayerOutputShapeCalculator()
output_shape = calculator.compute_conv_output_shape(input_shape, 
                                                    filter_shape, 
                                                    n_filters, 
                                                    stride, 
                                                    padding_size)

print('Conv Output Shape: {}'.format(output_shape))

Input Shape: [32, 32, 3]
Filter Shape: [8, 8, 3]
Num Filters: 20

Conv Height: 14
Conv Width: 14
Conv Depth: 20

Conv Output Shape: (14, 14, 20)


## ConvLayerOutputShapeDemo

In [5]:
class ConvLayerOutputShapeDemo:
    
    def run(self, input_shape, filter_shape, n_filters, stride, padding='SAME'):        
        input = tf.placeholder(tf.float32,
                               (None, input_shape[0], input_shape[1], input_shape[2]))
        
        print('Input Shape: {}'.format(input.shape))
        print('Filter Shape: {}'.format((filter_shape[0], filter_shape[1])))
        print('Output Depth: {}'.format(n_filters))
        print('')
        
        conv_weights = tf.Variable(tf.truncated_normal(
            (filter_shape[0], filter_shape[1], input_shape[2], n_filters)))
        conv_bias = tf.Variable(tf.zeros(n_filters))
        
        print('Conv Weights Shape: {}'.format(conv_weights.shape))
        print('Conv Bias Shape: {}'.format(conv_bias.shape))
        print('')
        
        conv_strides = [1, stride, stride, 1] # (batch, height, width, depth)
        
        print('Conv Strides: {}'.format(conv_strides))
        print('')
        
        conv = tf.nn.conv2d(input, conv_weights, conv_strides, padding) + conv_bias
        
        print('Conv Output Shape ({}): {}'.format(padding, conv.shape))

In [6]:
demo = ConvLayerOutputShapeDemo()
demo.run(input_shape, filter_shape, n_filters, stride, padding='VALID')

Input Shape: (?, 32, 32, 3)
Filter Shape: (8, 8)
Output Depth: 20

Conv Weights Shape: (8, 8, 3, 20)
Conv Bias Shape: (20,)

Conv Strides: [1, 2, 2, 1]

Conv Output Shape (VALID): (?, 13, 13, 20)


In [7]:
demo.run(input_shape, filter_shape, n_filters, stride, padding='SAME')

Input Shape: (?, 32, 32, 3)
Filter Shape: (8, 8)
Output Depth: 20

Conv Weights Shape: (8, 8, 3, 20)
Conv Bias Shape: (20,)

Conv Strides: [1, 2, 2, 1]

Conv Output Shape (SAME): (?, 16, 16, 20)


## ConvLayerParamNumCalculator

In [10]:
class ConvLayerParamNumCalculator:
    
    def compute_num_params_no_param_sharing(self,
                                            input_shape,
                                            filter_shape,
                                            n_filters,
                                            stride,
                                            p_size):
        
        num_filter_neurons = self.compute_num_filter_neurons(filter_shape)
        num_output_neurons = self.compute_num_output_neurons(input_shape, 
                                                             filter_shape, 
                                                             n_filters, 
                                                             stride, 
                                                             p_size)
        return int(num_filter_neurons * num_output_neurons)
    
    def compute_num_params_with_param_sharing(self,
                                              input_shape,
                                              filter_shape,
                                              n_filters):
        
        num_filter_neurons = self.compute_num_filter_neurons(filter_shape)
        num_channels = n_filters
        
        return num_filter_neurons * num_channels
        
    def compute_num_filter_neurons(self, filter_shape):
        num_weights = filter_shape[0] * filter_shape[1] * filter_shape[2]
        num_biases = 1
        return num_weights + num_biases
    
    def compute_num_output_neurons(self, input_shape, filter_shape, n_filters, stride, p_size):
        output_shape = ConvLayerOutputShapeCalculator()\
            .compute_conv_output_shape(input_shape, 
                                       filter_shape,
                                       n_filters,
                                       stride,
                                       p_size)
        return output_shape[0] * output_shape[1] * output_shape[2]

In [11]:
calculator = ConvLayerParamNumCalculator()

num_params = calculator.compute_num_params_no_param_sharing(input_shape, 
                                                            filter_shape, 
                                                            n_filters,
                                                            stride, 
                                                            padding_size)

print('Number of Parameters w/o Parameter Sharing: {}\n'.format(num_params))

num_params = calculator.compute_num_params_with_param_sharing(input_shape,
                                                              filter_shape,
                                                              n_filters)

print('Number of Paramters with Parameter Sharing: {}\n'.format(num_params))

Input Shape: [32, 32, 3]
Filter Shape: [8, 8, 3]
Num Filters: 20

Conv Height: 14
Conv Width: 14
Conv Depth: 20

Number of Parameters w/o Parameter Sharing: 756560

Number of Paramters with Parameter Sharing: 3860



## PoolLayerOutputShapeCalculator

In [12]:
class PoolLayerOutputShapeCalculator:
    
    def compute_output_shape(self, input_shape, filter_shape, stride):
        output_h = self.compute_output_height(input_shape, filter_shape, stride)
        output_w = self.compute_output_width(input_shape, filter_shape, stride)
        output_d = input_shape[2]
        return (output_h, output_w, output_d)
        
    def compute_output_height(self, input_shape, filter_shape, stride):
        return int((input_shape[0]-filter_shape[0])/stride) + 1
    
    def compute_output_width(self, input_shape, filter_shape, stride):
        return int((input_shape[1]-filter_shape[1])/stride) + 1

In [13]:
input_shape = [4, 4, 5]
filter_shape = [2, 2]
stride = 2

calculator = PoolLayerOutputShapeCalculator()
output_shape = calculator.compute_output_shape(input_shape, filter_shape, stride)

print('Pool Layer Output Shape: {}'.format(output_shape))

Pool Layer Output Shape: (2, 2, 5)


## PoolLayerOutputShapeDemo

In [14]:
class PoolLayerOutputShapeDemo:
    
    def run(self, input_shape, filter_shape, stride, padding='SAME'):
        input = tf.placeholder(tf.float32, (None, *input_shape))
        
        print('Input Shape: {}'.format(input.shape))
        print('')
        
        filters = [1, *filter_shape, 1]
        strides = [1, stride, stride, 1]
        
        print('Filters: {}'.format(filters))
        print('Strides: {}'.format(strides))
        print('')
        
        pool = tf.nn.max_pool(input, filters, strides, padding)
        
        print('Pool Output Shape ({}): {}'.format(padding, pool.shape))

In [15]:
demo = PoolLayerOutputShapeDemo()
demo.run(input_shape, filter_shape, stride, 'SAME')

Input Shape: (?, 4, 4, 5)

Filters: [1, 2, 2, 1]
Strides: [1, 2, 2, 1]

Pool Output Shape (SAME): (?, 2, 2, 5)


In [16]:
demo.run(input_shape, filter_shape, stride, 'VALID')

Input Shape: (?, 4, 4, 5)

Filters: [1, 2, 2, 1]
Strides: [1, 2, 2, 1]

Pool Output Shape (VALID): (?, 2, 2, 5)


## ConvNetwork

In [17]:
class ConvNetwork:
    
    def build(self, 
              input_shape,
              conv_1_filter_shape,
              conv_1_n_outputs,
              conv_2_filter_shape,
              conv_2_n_outputs,
              fc_dim,
              fc_n_outputs,
              out_n_classes,
              dropout,
              learning_rate):
        
        self.x, self.y, self.keep_prob = self.create_placeholders(input_shape, out_n_classes)
        
        self.weights, self.biases = self.create_weights_and_biases(conv_1_filter_shape,
                                                                   conv_1_n_outputs,
                                                                   conv_2_filter_shape,
                                                                   conv_2_n_outputs,
                                                                   fc_dim,
                                                                   fc_n_outputs,
                                                                   out_n_classes)
        
        self.logits = self.build_layers(self.x, self.weights, self.biases, self.keep_prob)
        
        self.cost, self.optimizer = self.define_cost_and_optimizer(self.logits, 
                                                                   self.y, 
                                                                   learning_rate)
        
        self.accuracy = self.define_accuracy(self.logits, self.y)
        
    def create_placeholders(self, input_shape, out_n_classes):
        x = tf.placeholder(tf.float32, [None, *input_shape])
        y = tf.placeholder(tf.float32, [None, out_n_classes])
        keep_prob = tf.placeholder(tf.float32)
        return x, y, keep_prob
 
    def create_weights_and_biases(self,
                                  conv_1_filter_shape,
                                  conv_1_n_outputs,
                                  conv_2_filter_shape,
                                  conv_2_n_outputs,
                                  fc_dim,
                                  fc_n_outputs,
                                  out_n_classes):
        
        weights = {
            'wc1': tf.Variable(tf.random_normal([*conv_1_filter_shape, conv_1_n_outputs])),
            'wc2': tf.Variable(tf.random_normal([*conv_2_filter_shape, conv_2_n_outputs])),
            'wfc': tf.Variable(tf.random_normal([fc_dim, fc_n_outputs])),
            'out': tf.Variable(tf.random_normal([fc_n_outputs, out_n_classes]))
        }
        
        biases = {
            'bc1': tf.Variable(tf.random_normal([conv_1_n_outputs])),
            'bc2': tf.Variable(tf.random_normal([conv_2_n_outputs])),
            'bfc': tf.Variable(tf.random_normal([fc_n_outputs])),
            'out': tf.Variable(tf.random_normal([out_n_classes]))
        }
        
        return weights, biases
 
    def build_layers(self, x, weights, biases, dropout):
        # Conv layer 1 - 28*28*1 to 14*14*32
        conv1 = self.get_conv2d_layer(x, weights['wc1'], biases['bc1'])
        conv1 = self.get_maxpool2d_layer(conv1, k=2)

        # Conv layer 2 - 14*14*32 to 7*7*64
        conv2 = self.get_conv2d_layer(conv1, weights['wc2'], biases['bc2'])
        conv2 = self.get_maxpool2d_layer(conv2, k=2)

        # Fully-connected layer - 7*7*64 to 1024
        fc = self.get_fc_layer(conv2, weights['wfc'], biases['bfc'], dropout)
        
        # Output Layer - class prediction - 1024 to 10
        out = self.get_output_layer(fc, weights['out'], biases['out'])
        
        return out
    
    def get_conv2d_layer(self, x, w, b, s=1):
        x = tf.nn.conv2d(x, w, strides=[1, s, s, 1], padding='SAME')
        x = tf.nn.bias_add(x, b)
        return tf.nn.relu(x)
    
    def get_maxpool2d_layer(self, x, k=2):
        return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')
    
    def get_fc_layer(self, x, w, b, dropout):
        x = tf.reshape(x, [-1, w.get_shape().as_list()[0]])
        x = tf.add(tf.matmul(x, w), b)
        x = tf.nn.relu(x)
        return tf.nn.dropout(x, dropout)
    
    def get_output_layer(self, x, w, b):
        return tf.add(tf.matmul(x, w), b)
    
    def define_cost_and_optimizer(self, logits, y, learning_rate):
        cost = tf.reduce_mean(\
                              tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
            .minimize(cost)
        return cost, optimizer

    def define_accuracy(self, logits, y):
        correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
        return tf.reduce_mean(tf.cast(correct_pred, tf.float32))

## ConvNetworkTrainer

In [18]:
class ConvNetworkTrainer:
    
    def train_network(self, 
                      network, 
                      mnist,
                      epochs=10,
                      batch_size=128,
                      test_valid_size=256,
                      dropout=0.75):
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
                
            for epoch in range(epochs):

                for batch_idx in range(mnist.train.num_examples//batch_size):
                    batch_x, batch_y = mnist.train.next_batch(batch_size)

                    sess.run(network.optimizer,
                             feed_dict={network.x: batch_x,
                                        network.y: batch_y,
                                        network.keep_prob: dropout})

                    # Calculate batch loss and validation accuracy
                    loss = sess.run(network.cost,
                                    feed_dict={network.x: batch_x,
                                               network.y: batch_y,
                                               network.keep_prob: 1.})

                    valid_acc = sess.run(network.accuracy,
                                         feed_dict={
                                            network.x: mnist.validation.images[:test_valid_size],
                                            network.y: mnist.validation.labels[:test_valid_size],
                                            network.keep_prob: 1.})

                    logStr = 'Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'

                    print(logStr.format(epoch+1, batch_idx+1, loss, valid_acc))
                          
            # Calculate test accuracy
            test_acc = sess.run(network.accuracy,
                                feed_dict={
                                    network.x: mnist.test.images[:test_valid_size],
                                    network.y: mnist.test.labels[:test_valid_size],
                                    network.keep_prob: 1.})   
                          
            print('Testing Accuracy: {}'.format(test_acc))

In [19]:
input_shape = [28, 28, 1]
conv_1_filter_shape = [5, 5, 1]
conv_1_n_outputs = 32
conv_2_filter_shape = [5, 5, 32]
conv_2_n_outputs = 64
fc_dim = 7*7*64
fc_n_outputs = 1024
out_n_classes = 10
dropout = 0.75
learning_rate = 0.00001
epochs = 10
batch_size = 128
test_valid_size = 256

In [20]:
network = ConvNetwork()

network.build(input_shape,
              conv_1_filter_shape,
              conv_1_n_outputs,
              conv_2_filter_shape,
              conv_2_n_outputs,
              fc_dim,
              fc_n_outputs,
              out_n_classes,
              dropout,
              learning_rate)

In [21]:
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz


In [22]:
networkTrainer = ConvNetworkTrainer()

networkTrainer.train_network(network,
                             mnist,
                             epochs,
                             batch_size,
                             test_valid_size,
                             dropout)

Epoch  1, Batch   1 - Loss: 46034.9180 Validation Accuracy: 0.039062
Epoch  1, Batch   2 - Loss: 40038.8555 Validation Accuracy: 0.070312
Epoch  1, Batch   3 - Loss: 36304.4609 Validation Accuracy: 0.078125
Epoch  1, Batch   4 - Loss: 37764.7969 Validation Accuracy: 0.082031
Epoch  1, Batch   5 - Loss: 31106.7773 Validation Accuracy: 0.097656
Epoch  1, Batch   6 - Loss: 31976.8789 Validation Accuracy: 0.082031
Epoch  1, Batch   7 - Loss: 28067.0625 Validation Accuracy: 0.117188
Epoch  1, Batch   8 - Loss: 29678.0938 Validation Accuracy: 0.117188
Epoch  1, Batch   9 - Loss: 28386.7930 Validation Accuracy: 0.125000
Epoch  1, Batch  10 - Loss: 23891.8281 Validation Accuracy: 0.121094
Epoch  1, Batch  11 - Loss: 23187.0469 Validation Accuracy: 0.117188
Epoch  1, Batch  12 - Loss: 23650.0664 Validation Accuracy: 0.148438
Epoch  1, Batch  13 - Loss: 20978.7480 Validation Accuracy: 0.156250
Epoch  1, Batch  14 - Loss: 24865.6211 Validation Accuracy: 0.164062
Epoch  1, Batch  15 - Loss: 21044.

Epoch  1, Batch 120 - Loss:  3656.0056 Validation Accuracy: 0.652344
Epoch  1, Batch 121 - Loss:  4366.3057 Validation Accuracy: 0.652344
Epoch  1, Batch 122 - Loss:  5072.8750 Validation Accuracy: 0.652344
Epoch  1, Batch 123 - Loss:  3356.4531 Validation Accuracy: 0.652344
Epoch  1, Batch 124 - Loss:  4002.2305 Validation Accuracy: 0.652344
Epoch  1, Batch 125 - Loss:  3484.3008 Validation Accuracy: 0.656250
Epoch  1, Batch 126 - Loss:  3241.3501 Validation Accuracy: 0.660156
Epoch  1, Batch 127 - Loss:  4128.2236 Validation Accuracy: 0.660156
Epoch  1, Batch 128 - Loss:  3710.9275 Validation Accuracy: 0.660156
Epoch  1, Batch 129 - Loss:  4213.7271 Validation Accuracy: 0.664062
Epoch  1, Batch 130 - Loss:  5445.9082 Validation Accuracy: 0.667969
Epoch  1, Batch 131 - Loss:  3784.0750 Validation Accuracy: 0.671875
Epoch  1, Batch 132 - Loss:  4231.2783 Validation Accuracy: 0.667969
Epoch  1, Batch 133 - Loss:  3768.0781 Validation Accuracy: 0.671875
Epoch  1, Batch 134 - Loss:  3484.

Epoch  1, Batch 239 - Loss:  1762.9729 Validation Accuracy: 0.714844
Epoch  1, Batch 240 - Loss:  2432.2642 Validation Accuracy: 0.714844
Epoch  1, Batch 241 - Loss:  1659.2363 Validation Accuracy: 0.710938
Epoch  1, Batch 242 - Loss:  2814.0486 Validation Accuracy: 0.714844
Epoch  1, Batch 243 - Loss:  2827.6643 Validation Accuracy: 0.714844
Epoch  1, Batch 244 - Loss:  2093.5308 Validation Accuracy: 0.730469
Epoch  1, Batch 245 - Loss:  2351.9351 Validation Accuracy: 0.722656
Epoch  1, Batch 246 - Loss:  2002.2316 Validation Accuracy: 0.734375
Epoch  1, Batch 247 - Loss:  2098.4917 Validation Accuracy: 0.730469
Epoch  1, Batch 248 - Loss:  2161.9797 Validation Accuracy: 0.734375
Epoch  1, Batch 249 - Loss:  2315.6470 Validation Accuracy: 0.734375
Epoch  1, Batch 250 - Loss:  1886.4583 Validation Accuracy: 0.734375
Epoch  1, Batch 251 - Loss:  2006.8148 Validation Accuracy: 0.734375
Epoch  1, Batch 252 - Loss:  2152.6521 Validation Accuracy: 0.734375
Epoch  1, Batch 253 - Loss:  2537.

Epoch  1, Batch 358 - Loss:  1891.2097 Validation Accuracy: 0.761719
Epoch  1, Batch 359 - Loss:  1406.1089 Validation Accuracy: 0.757812
Epoch  1, Batch 360 - Loss:   860.8077 Validation Accuracy: 0.765625
Epoch  1, Batch 361 - Loss:  1857.5747 Validation Accuracy: 0.761719
Epoch  1, Batch 362 - Loss:  2417.6929 Validation Accuracy: 0.757812
Epoch  1, Batch 363 - Loss:  1407.5007 Validation Accuracy: 0.757812
Epoch  1, Batch 364 - Loss:  1895.1926 Validation Accuracy: 0.761719
Epoch  1, Batch 365 - Loss:  1412.7247 Validation Accuracy: 0.761719
Epoch  1, Batch 366 - Loss:  2485.0317 Validation Accuracy: 0.757812
Epoch  1, Batch 367 - Loss:  1865.8013 Validation Accuracy: 0.757812
Epoch  1, Batch 368 - Loss:  1897.9683 Validation Accuracy: 0.757812
Epoch  1, Batch 369 - Loss:  1630.1680 Validation Accuracy: 0.761719
Epoch  1, Batch 370 - Loss:  2449.1191 Validation Accuracy: 0.765625
Epoch  1, Batch 371 - Loss:  2218.5356 Validation Accuracy: 0.765625
Epoch  1, Batch 372 - Loss:  2557.

Epoch  2, Batch  48 - Loss:  1569.1415 Validation Accuracy: 0.773438
Epoch  2, Batch  49 - Loss:  1671.2161 Validation Accuracy: 0.765625
Epoch  2, Batch  50 - Loss:  1968.5972 Validation Accuracy: 0.757812
Epoch  2, Batch  51 - Loss:  1671.8591 Validation Accuracy: 0.757812
Epoch  2, Batch  52 - Loss:  1463.0405 Validation Accuracy: 0.761719
Epoch  2, Batch  53 - Loss:  1457.5510 Validation Accuracy: 0.761719
Epoch  2, Batch  54 - Loss:  1144.7487 Validation Accuracy: 0.765625
Epoch  2, Batch  55 - Loss:  1508.3198 Validation Accuracy: 0.761719
Epoch  2, Batch  56 - Loss:  1534.1035 Validation Accuracy: 0.761719
Epoch  2, Batch  57 - Loss:  1605.0902 Validation Accuracy: 0.761719
Epoch  2, Batch  58 - Loss:  1723.4741 Validation Accuracy: 0.765625
Epoch  2, Batch  59 - Loss:  1198.7642 Validation Accuracy: 0.761719
Epoch  2, Batch  60 - Loss:  1352.1228 Validation Accuracy: 0.761719
Epoch  2, Batch  61 - Loss:  1322.7776 Validation Accuracy: 0.761719
Epoch  2, Batch  62 - Loss:  1058.

...

Epoch  9, Batch 377 - Loss:   542.6168 Validation Accuracy: 0.882812
Epoch  9, Batch 378 - Loss:   387.1367 Validation Accuracy: 0.882812
Epoch  9, Batch 379 - Loss:   359.9219 Validation Accuracy: 0.882812
Epoch  9, Batch 380 - Loss:   189.8110 Validation Accuracy: 0.886719
Epoch  9, Batch 381 - Loss:   280.3832 Validation Accuracy: 0.886719
Epoch  9, Batch 382 - Loss:   264.0540 Validation Accuracy: 0.886719
Epoch  9, Batch 383 - Loss:   403.1322 Validation Accuracy: 0.886719
Epoch  9, Batch 384 - Loss:   218.7781 Validation Accuracy: 0.882812
Epoch  9, Batch 385 - Loss:    87.1952 Validation Accuracy: 0.878906
Epoch  9, Batch 386 - Loss:   160.5626 Validation Accuracy: 0.875000
Epoch  9, Batch 387 - Loss:   227.8875 Validation Accuracy: 0.875000
Epoch  9, Batch 388 - Loss:   266.9024 Validation Accuracy: 0.875000
Epoch  9, Batch 389 - Loss:   174.5020 Validation Accuracy: 0.875000
Epoch  9, Batch 390 - Loss:   309.5414 Validation Accuracy: 0.875000
Epoch  9, Batch 391 - Loss:   408.

Epoch 10, Batch  67 - Loss:   162.2393 Validation Accuracy: 0.882812
Epoch 10, Batch  68 - Loss:   246.9514 Validation Accuracy: 0.882812
Epoch 10, Batch  69 - Loss:   417.5258 Validation Accuracy: 0.882812
Epoch 10, Batch  70 - Loss:   254.8509 Validation Accuracy: 0.882812
Epoch 10, Batch  71 - Loss:   273.3278 Validation Accuracy: 0.890625
Epoch 10, Batch  72 - Loss:   327.6089 Validation Accuracy: 0.886719
Epoch 10, Batch  73 - Loss:   230.5223 Validation Accuracy: 0.890625
Epoch 10, Batch  74 - Loss:   310.2911 Validation Accuracy: 0.890625
Epoch 10, Batch  75 - Loss:   370.8249 Validation Accuracy: 0.894531
Epoch 10, Batch  76 - Loss:   276.5089 Validation Accuracy: 0.890625
Epoch 10, Batch  77 - Loss:   165.8059 Validation Accuracy: 0.890625
Epoch 10, Batch  78 - Loss:   204.8972 Validation Accuracy: 0.886719
Epoch 10, Batch  79 - Loss:   304.2057 Validation Accuracy: 0.886719
Epoch 10, Batch  80 - Loss:   223.4865 Validation Accuracy: 0.882812
Epoch 10, Batch  81 - Loss:   249.

Epoch 10, Batch 186 - Loss:   388.2245 Validation Accuracy: 0.878906
Epoch 10, Batch 187 - Loss:   239.5736 Validation Accuracy: 0.878906
Epoch 10, Batch 188 - Loss:   259.9196 Validation Accuracy: 0.875000
Epoch 10, Batch 189 - Loss:   305.4857 Validation Accuracy: 0.875000
Epoch 10, Batch 190 - Loss:   215.9266 Validation Accuracy: 0.875000
Epoch 10, Batch 191 - Loss:   246.9017 Validation Accuracy: 0.875000
Epoch 10, Batch 192 - Loss:   280.8763 Validation Accuracy: 0.875000
Epoch 10, Batch 193 - Loss:   264.2707 Validation Accuracy: 0.875000
Epoch 10, Batch 194 - Loss:   304.0902 Validation Accuracy: 0.875000
Epoch 10, Batch 195 - Loss:   188.3892 Validation Accuracy: 0.875000
Epoch 10, Batch 196 - Loss:   293.2325 Validation Accuracy: 0.875000
Epoch 10, Batch 197 - Loss:   315.1639 Validation Accuracy: 0.875000
Epoch 10, Batch 198 - Loss:   143.3292 Validation Accuracy: 0.875000
Epoch 10, Batch 199 - Loss:   321.2767 Validation Accuracy: 0.875000
Epoch 10, Batch 200 - Loss:   173.

Epoch 10, Batch 305 - Loss:   262.5280 Validation Accuracy: 0.878906
Epoch 10, Batch 306 - Loss:   254.1494 Validation Accuracy: 0.875000
Epoch 10, Batch 307 - Loss:   296.3698 Validation Accuracy: 0.878906
Epoch 10, Batch 308 - Loss:   322.1735 Validation Accuracy: 0.878906
Epoch 10, Batch 309 - Loss:   228.2536 Validation Accuracy: 0.878906
Epoch 10, Batch 310 - Loss:   225.1440 Validation Accuracy: 0.878906
Epoch 10, Batch 311 - Loss:   372.5031 Validation Accuracy: 0.878906
Epoch 10, Batch 312 - Loss:   244.0405 Validation Accuracy: 0.878906
Epoch 10, Batch 313 - Loss:   261.6147 Validation Accuracy: 0.875000
Epoch 10, Batch 314 - Loss:   334.4716 Validation Accuracy: 0.875000
Epoch 10, Batch 315 - Loss:   235.8839 Validation Accuracy: 0.875000
Epoch 10, Batch 316 - Loss:   199.2065 Validation Accuracy: 0.878906
Epoch 10, Batch 317 - Loss:   283.0274 Validation Accuracy: 0.875000
Epoch 10, Batch 318 - Loss:   152.7902 Validation Accuracy: 0.875000
Epoch 10, Batch 319 - Loss:   312.

Epoch 10, Batch 424 - Loss:   216.9382 Validation Accuracy: 0.878906
Epoch 10, Batch 425 - Loss:   274.3071 Validation Accuracy: 0.878906
Epoch 10, Batch 426 - Loss:   203.4363 Validation Accuracy: 0.878906
Epoch 10, Batch 427 - Loss:   218.9312 Validation Accuracy: 0.878906
Epoch 10, Batch 428 - Loss:   287.3615 Validation Accuracy: 0.878906
Epoch 10, Batch 429 - Loss:   199.9969 Validation Accuracy: 0.878906
Testing Accuracy: 0.859375


## FilterShapeCalculator

In [23]:
class FilterShapeCalculator:
    
    def compute_filter_shape(self, input_shape, output_shape, strides):
        filter_height = None
        filter_width = None
        
        for f_h in range(input_shape[1], 1, -1):
            # out_height = ceil(float(in_height - filter_height + 1) / float(strides[1]))
            out_h = math.ceil(float(input_shape[1] - f_h + 1) / float(strides[1]))

            if out_h == output_shape[1]:
                filter_height = f_h
                break
                
        for f_w in range(input_shape[2], 1, -1):
            # out_width = ceil(float(in_width - filter_width + 1) / float(strides[2]))
            out_w = math.ceil(float(input_shape[2] - f_w + 1) / float(strides[2]))
            
            if out_w == output_shape[2]:
                filter_width = f_w
                break
                
        return (filter_height, filter_width)

In [24]:
input_shape = (1, 4, 4, 1)
output_shape = (1, 2, 2, 3)
strides = (1, 2, 2, 1)

calculator = FilterShapeCalculator()
filter_shape = calculator.compute_filter_shape(input_shape, output_shape, strides)
print(filter_shape)

(2, 2)


## TensorflowConvLayer

In [25]:
class TensorflowConvLayer:
    
    def conv2d(self, input, input_shape, output_shape):
        """
        Setup the strides, padding and filter weight/bias such that
        the output shape is (1, 2, 2, 3).
        """
        
        # Set the stride for each dimension (batch_size, height, width, depth)
        strides = [1, 2, 2, 1]
        
        calculator = FilterShapeCalculator()
        filter_shape = calculator.compute_filter_shape(input_shape, output_shape, strides)
        
        # Filter (weights and bias)
        # The shape of the filter weight is (height, width, input_depth, output_depth)
        # The shape of the filter bias is (output_depth,)
        filter_weights = tf.Variable(tf.truncated_normal((filter_shape[0], 
                                                          filter_shape[1], 
                                                          input_shape[3], 
                                                          output_shape[3])))
        filter_biases = tf.Variable(tf.zeros(output_shape[3]))

        # Set the padding, either 'VALID' or 'SAME'
        padding = 'VALID'
        
        return tf.add(tf.nn.conv2d(input, filter_weights, strides, padding), filter_biases)

In [26]:
# tf.nn.conv2d() requires the input be 4D (batch_size, height, width, depth)
# (1, 4, 4, 1)
x = np.array([
    [0, 1, 0.5, 10],
    [2, 2.5, 1, -8],
    [4, 0, 5, 6],
    [15, 1, 2, 3]], dtype=np.float32).reshape(input_shape)

X = tf.constant(x)

convLayer = TensorflowConvLayer()
conv_output = convLayer.conv2d(X, input_shape, output_shape)

print(conv_output.shape)

(1, 2, 2, 3)


## TensorflowPoolingLayer

In [27]:
class TensorflowPoolingLayer:
    
    def maxpool2d(self, input, input_shape, output_shape):
        """
        Set the values to `strides` and `ksize` such that
        the output shape after pooling is (1, 2, 2, 1).
        """
        
        # Set the stride for each dimension (batch_size, height, width, depth)
        strides = [1, 2, 2, 1]
        
        # Set the ksize (filter size) for each dimension (batch_size, height, width, depth)
        calculator = FilterShapeCalculator()
        filter_shape = calculator.compute_filter_shape(input_shape, output_shape, strides)
        ksize = [1, filter_shape[0], filter_shape[1], 1]
        
        # Set the padding, either 'VALID' or 'SAME'.
        padding = 'VALID'
        
        return tf.nn.max_pool(input, ksize, strides, padding)

In [28]:
output_shape = (1, 2, 2, 1)

poolLayer = TensorflowPoolingLayer()
pool_output = poolLayer.maxpool2d(X, input_shape, output_shape)

print(pool_output.shape)

(1, 2, 2, 1)
